In [ ]:
!pip install segmentation_models_pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 13.2 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from torchvision import transforms
import torchvision.transforms.functional as TF # Added for manual augmentations
from tqdm import tqdm
import segmentation_models_pytorch as smp
from google.colab import drive
import random
import matplotlib.pyplot as plt

torch.manual_seed(42)
random.seed(42)
np.random.seed(42)
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)
torch.backends.cudnn.deterministic = True
g = torch.Generator()
g.manual_seed(42)

In [ ]:
# --- CONFIGURATION ---
DATASET_NAME = "lakshaymiddha/crack-segmentation-dataset"
WEIGHTS = "imagenet"
IMG_SIZE = 448
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# --- DATASET DEFINITION ---
class CrackDataset(Dataset):
    def __init__(self, root_dir, split="train", filter_non_crack=False):
        self.root_dir = root_dir
        self.split = split

        self.images_dir = os.path.join(root_dir, split, "images")
        self.masks_dir = os.path.join(root_dir, split, "masks")

        # Get all JPGs
        all_files = sorted([
            f for f in os.listdir(self.images_dir)
            if f.lower().endswith('.jpg')
        ])

        # EXPERIMENTAL: Filter out images with no cracks
        # Set to False to train on the full dataset (helps reduce false positives)
        if filter_non_crack:
            self.image_filenames = [
                f for f in all_files
                if "noncrack" not in f.lower()
            ]
            filtered_count = len(all_files) - len(self.image_filenames)
            print(f"[{split}] Filtering Enabled: Removed {filtered_count} non-crack images.")
        else:
            self.image_filenames = all_files

        # Define Normalization (Standard ImageNet stats)
        self.normalize = transforms.Normalize(
            mean=(0.485, 0.456, 0.406),
            std=(0.229, 0.224, 0.225)
        )

        print(f"[{split}] Final Dataset Size: {len(self.image_filenames)} images.")

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_name = self.image_filenames[idx]
        img_path = os.path.join(self.images_dir, img_name)
        mask_path = os.path.join(self.masks_dir, img_name)

        # 1. Load Data
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if mask is None:
            mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

        # 2. Resize
        # Resize both to target size
        # interpolation=cv2.INTER_NEAREST is crucial for masks to keep 0/255 values
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        mask = cv2.resize(mask, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)

        # 3. Convert to Tensor (Normalized 0-1)
        # ToTensor converts (H, W, C) -> (C, H, W) and scales 0-255 to 0.0-1.0
        image = TF.to_tensor(image)
        mask = torch.from_numpy(mask).float().unsqueeze(0) / 255.0 # (1, H, W) range 0.0-1.0

        # 4. Augmentations (Train Only)
        if self.split == "train":
            # A. Random Horizontal Flip
            if random.random() > 0.5:
                image = TF.hflip(image)
                mask = TF.hflip(mask)

            # B. Random Vertical Flip
            if random.random() > 0.5:
                image = TF.vflip(image)
                mask = TF.vflip(mask)

            # C. Random Rotation (90, 180, 270)
            if random.random() > 0.5:
                angle = random.choice([90, 180, 270])
                image = TF.rotate(image, angle)
                mask = TF.rotate(mask, angle)

            # D. Color Jitter (Image Only)
            if random.random() > 0.2: # Apply 80% of the time
                brightness_factor = random.uniform(0.8, 1.2)
                image = TF.adjust_brightness(image, brightness_factor)
                contrast_factor = random.uniform(0.8, 1.2)
                image = TF.adjust_contrast(image, contrast_factor)

        # 5. Final Normalization (Image Only)
        image = self.normalize(image)

        # Binarize mask
        mask = (mask > 0.5).float()

        return image, mask


In [ ]:
# --- TRAINING UTILS ---
def train_fn(loader, model, optimizer, loss_fn, scaler):
    model.train()
    loop = tqdm(loader)
    epoch_loss = 0

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(DEVICE)
        targets = targets.to(DEVICE)

        with autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        epoch_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    return epoch_loss / len(loader)

In [ ]:
def check_accuracy(loader, model, device="cuda"):
    dice_scores = []
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            with autocast():
                preds = torch.sigmoid(model(x))
                preds = (preds > 0.5).float()

            preds = preds.view(preds.size(0), -1)
            targets = y.view(y.size(0), -1)

            intersection = (preds * targets).sum(dim=1)
            dice = (2. * intersection + 1e-8) / (preds.sum(dim=1) + targets.sum(dim=1) + 1e-8)
            dice_scores.extend(dice.cpu().numpy())

    return sum(dice_scores) / len(dice_scores)

In [ ]:
def visualize_predictions(loader, model, save_folder, encoder_name, file_suffix, device="cuda"):
    """
    Saves a grid of predictions (Input | Truth | Prediction) to disk.
    Uses fixed hardcoded indices to ensure 100% consistent comparisons across runs.
    """
    print("Generating visualization...")
    model.eval()

    # Use the underlying dataset directly to get consistent samples
    dataset = loader.dataset

    # HARDCODED INDICES
    # We select specific indices to ensure the exact same images are visualized
    # every time, regardless of dataset resizing or batch shuffle.
    # These indices are spaced out to likely cover different crack types.
    indices = [10, 100, 250, 400, 550, 700]

    # Safety check: ensure indices exist in the dataset (in case test set is small)
    indices = [i for i in indices if i < len(dataset)]
    if not indices:
        indices = list(range(min(6, len(dataset))))

    num_samples = len(indices)

    # Manually construct the batch
    images_list = []
    masks_list = []

    for idx in indices:
        image, mask = dataset[idx]
        images_list.append(image)
        masks_list.append(mask)

    # Stack tensors to create batch
    images = torch.stack(images_list).to(device)
    masks = torch.stack(masks_list).cpu()

    with torch.no_grad():
        with autocast():
            preds = torch.sigmoid(model(images))
            preds = (preds > 0.5).float()

    # Move predictions to CPU
    preds = preds.cpu()
    images = images.cpu()

    # ImageNet stats for denormalization
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)

    # Plotting
    plt.figure(figsize=(15, 5 * num_samples))
    plt.suptitle(f"Model: {encoder_name}", fontsize=20) # Add global title for easy identification

    for i in range(num_samples):
        # Denormalize Image
        img = images[i] * std + mean
        img = torch.clamp(img, 0, 1)
        img = img.permute(1, 2, 0).numpy()

        mask = masks[i].squeeze().numpy()
        pred = preds[i].squeeze().numpy()

        # 1. Input Image
        plt.subplot(num_samples, 3, i*3 + 1)
        plt.imshow(img)
        plt.title(f"Input Image #{indices[i]}")
        plt.axis("off")

        # 2. Ground Truth
        plt.subplot(num_samples, 3, i*3 + 2)
        plt.imshow(mask, cmap="gray")
        plt.title("Ground Truth")
        plt.axis("off")

        # 3. Prediction
        plt.subplot(num_samples, 3, i*3 + 3)
        plt.imshow(pred, cmap="gray")
        plt.title(f"Prediction ({encoder_name})")
        plt.axis("off")

    # Use tight_layout to handle the new suptitle properly
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    save_path = os.path.join(save_folder, f"prediction_visuals_{file_suffix}.png")
    plt.savefig(save_path)
    plt.close()
    print(f"Visualization saved to: {save_path}")

In [ ]:
# --- MAIN EXECUTION ---
def main(encoder_name, batch_size, learning_rate,
         num_workers=16,
         patience=10,
         full_training_epochs=50,
         freeze_encoder_epochs=0,
         # 0.5 default to both alpha and beta is equivalent to the Dice loss
         tversky_alpha=0.5,
         tversky_beta=0.5,
         # Default learning rate for the encoder and the decoder parts are equal
         # but can be customized
         learning_rate_encoder=None,
         learning_rate_decoder=None):
    print(f"Setting up on {DEVICE}...")

    if learning_rate_encoder is None:
      learning_rate_encoder = learning_rate
    if learning_rate_decoder is None:
      learning_rate_decoder = learning_rate

    epochs = full_training_epochs + freeze_encoder_epochs

    file_suffix = f"{encoder_name}_ta_{tversky_alpha}_tb_{tversky_beta}_lre_{learning_rate_encoder}_lrd_{learning_rate_decoder}_fee_{freeze_encoder_epochs}_fte_{full_training_epochs}"

    # Mount Drive explicitly at the start for both API key and Checkpoints
    drive.mount('/content/drive')

    # Setup Kaggle
    if not os.path.exists("/root/.kaggle"):
        os.makedirs("/root/.kaggle")

    # Get kaggle.json ONLY from Drive
    kaggle_json_drive = "/content/drive/MyDrive/kaggle.json"

    if os.path.exists(kaggle_json_drive):
        print(f"Found kaggle.json in Drive at {kaggle_json_drive}")
        os.system(f"cp {kaggle_json_drive} /root/.kaggle/")
        os.system("chmod 600 /root/.kaggle/kaggle.json")
    else:
        print(f"WARNING: kaggle.json not found at {kaggle_json_drive}. Dataset download will likely fail.")

    # Download Dataset
    if not os.path.exists("crack-segmentation-dataset.zip"):
        print("Downloading dataset...")
        os.system(f"kaggle datasets download -d {DATASET_NAME}")
        os.system("unzip -q crack-segmentation-dataset.zip -d dataset")
        print("Dataset downloaded and extracted.")

    # Load Data
    train_ds = CrackDataset(
        root_dir="./dataset/crack_segmentation_dataset",
        split="train"
    )
    test_ds = CrackDataset(
        root_dir="./dataset/crack_segmentation_dataset",
        split="test"
    )

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        generator=g
    )
    # Shuffle Test Loader slightly so we see different samples in visualization every time
    test_loader = DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        generator=g
    )

    # Initialize Model
    model = smp.Unet(
        encoder_name=encoder_name,
        encoder_weights=WEIGHTS,
        in_channels=3,
        classes=1,
    ).to(DEVICE)

    # FREEZE ENCODER Logic (Initial Freeze)
    if freeze_encoder_epochs > 0:
        print(f"Freezing encoder parameters for the first {freeze_encoder_epochs} epochs...")
        for param in model.encoder.parameters():
            param.requires_grad = False

    loss_fn = smp.losses.TverskyLoss(mode="binary", alpha=tversky_alpha, beta=tversky_beta)
    optimizer = optim.Adam([
        {"params": model.encoder.parameters(), "lr": learning_rate_encoder},
        {"params": model.decoder.parameters(), "lr": learning_rate_decoder},
    ])
    scaler = GradScaler()

    # --- Checkpointing & Initialization ---
    # Drive is already mounted at the start
    CHECKPOINT_DIR = "/content/drive/MyDrive/CrackSegmentation_Checkpoints"
    if not os.path.exists(CHECKPOINT_DIR):
        os.makedirs(CHECKPOINT_DIR)
    CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, f"best_crack_model_{file_suffix}.pth")
    print(f"Checkpoints will be saved to: {CHECKPOINT_PATH}")

    best_dice = 0.0
    patience_counter = 0

    if os.path.exists(CHECKPOINT_PATH):
        print(f"Checkpoint found at {CHECKPOINT_PATH}! Loading existing model weights...")
        model.load_state_dict(torch.load(CHECKPOINT_PATH))

        # Evaluate baseline
        print("Evaluating loaded model...")
        best_dice = check_accuracy(test_loader, model, device=DEVICE)
        print(f"Loaded model Dice Score: {best_dice:.4f}")

    print(f"Starting training on {DEVICE} for {epochs} epochs...")

    try:
        freeze_encoder_epochs_start = freeze_encoder_epochs
        full_training = False if freeze_encoder_epochs > 0 else True
        epochs = full_training_epochs + freeze_encoder_epochs
        remaining_full_training_epochs = full_training_epochs
        for epoch in range(epochs):
            # Dynamic Unfreezing Logic
            if freeze_encoder_epochs_start > 0 and epoch == freeze_encoder_epochs_start:
                best_dice = 0.0
                patience_counter = 0
                print(f"Epoch {epoch+1}: Unfreezing encoder parameters for fine-tuning...")
                model.load_state_dict(torch.load(CHECKPOINT_PATH))
                for param in model.encoder.parameters():
                    param.requires_grad = True
                full_training = True

            if full_training:
              remaining_full_training_epochs -= 1
              if remaining_full_training_epochs == 0:
                break

            loss = train_fn(train_loader, model, optimizer, loss_fn, scaler)
            dice_score = check_accuracy(test_loader, model, device=DEVICE)

            print(f"Epoch {epoch+1}/{epochs} [{"Full training" if full_training else "Decoder-only training"}] | Loss: {loss:.4f} | Dice Score: {dice_score:.4f}")

            if dice_score > best_dice:
                best_dice = dice_score
                torch.save(model.state_dict(), CHECKPOINT_PATH)
                print(f"New Best Score! Saved Model to {CHECKPOINT_PATH} (Dice: {best_dice:.4f})")
                patience_counter = 0
            else:
                patience_counter += 1
                print(f"No improvement. Patience: {patience_counter}/{patience}")

                if patience_counter >= patience:
                  if epoch < freeze_encoder_epochs_start:
                    print("Early stopping triggered. Unfreezing encoder parameters.")
                    patience_counter = 0
                    freeze_encoder_epochs_start = epoch+1
                  else:
                    print(f"Early stopping triggered.")
                    break
    except KeyboardInterrupt:
        print("Training interrupted manually.")

    # --- VISUALIZATION ---
    # Generate visualization using the best model
    if os.path.exists(CHECKPOINT_PATH):
        print("Loading best model for visualization...")
        model.load_state_dict(torch.load(CHECKPOINT_PATH))
        visualize_predictions(test_loader, model, save_folder=CHECKPOINT_DIR, encoder_name=encoder_name, device=DEVICE, file_suffix=file_suffix)

    print("Training Complete.")

## resnet18

In [ ]:
# COMPLETE
main(
    encoder_name="resnet18",
    # Increased Batch Size to 128 for ResNet18 (Lightest network = Lowest VRAM usage)
    batch_size=128,
    # Scaled LR up because we increased Batch Size (Linear Scaling Rule)
    learning_rate=1e-3,
    full_training_epochs=50
)

Setting up on cuda...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found kaggle.json in Drive at /content/drive/MyDrive/kaggle.json
[train] Final Dataset Size: 9603 images.
[test] Final Dataset Size: 1695 images.


/tmp/ipython-input-3050472091.py:96: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Checkpoints will be saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet18_ta_0.5_tb_0.5_lre_0.001_lrd_0.001_fee_0_fte_50.pth
Starting training on cuda for 50 epochs...


  0%|          | 0/76 [00:00<?, ?it/s]/tmp/ipython-input-3144592320.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 76/76 [00:49<00:00,  1.53it/s, loss=0.933]
/tmp/ipython-input-3809429390.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/50 [Full training] | Loss: 0.8501 | Dice Score: 0.4181
New Best Score! Saved Model to /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet18_ta_0.5_tb_0.5_lre_0.001_lrd_0.001_fee_0_fte_50.pth (Dice: 0.4181)


...

Epoch 48/50 [Full training] | Loss: 0.2918 | Dice Score: 0.7074
No improvement. Patience: 8/10


100%|██████████| 76/76 [00:50<00:00,  1.51it/s, loss=0.35]


Epoch 49/50 [Full training] | Loss: 0.2916 | Dice Score: 0.7099
No improvement. Patience: 9/10
Loading best model for visualization...
Generating visualization...


/tmp/ipython-input-1775095349.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Visualization saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/prediction_visuals_resnet18_ta_0.5_tb_0.5_lre_0.001_lrd_0.001_fee_0_fte_50.png
Training Complete.


In [ ]:
# COMPLETE
main(
    encoder_name="resnet18",
    batch_size=128,
    learning_rate=1e-3,
    full_training_epochs=50,
    # First train the decoder for 50 epochs
    freeze_encoder_epochs=50
)

Setting up on cuda...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found kaggle.json in Drive at /content/drive/MyDrive/kaggle.json
[train] Final Dataset Size: 9603 images.
[test] Final Dataset Size: 1695 images.


/tmp/ipython-input-1728581891.py:96: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Freezing encoder parameters for the first 50 epochs...
Checkpoints will be saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet18_ta_0.5_tb_0.5_lre_0.001_lrd_0.001_fee_50_fte_50.pth
Starting training on cuda for 100 epochs...


  0%|          | 0/76 [00:00<?, ?it/s]/tmp/ipython-input-3144592320.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 76/76 [00:45<00:00,  1.68it/s, loss=0.84]
/tmp/ipython-input-3809429390.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/100 [Decoder-only training] | Loss: 0.8175 | Dice Score: 0.5412
New Best Score! Saved Model to /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet18_ta_0.5_tb_0.5_lre_0.001_lrd_0.001_fee_50_fte_50.pth (Dice: 0.5412)


...

Epoch 56/100 [Full training] | Loss: 0.3162 | Dice Score: 0.6703
No improvement. Patience: 9/10


100%|██████████| 76/76 [00:51<00:00,  1.49it/s, loss=0.276]


Epoch 57/100 [Full training] | Loss: 0.2978 | Dice Score: 0.6925
No improvement. Patience: 10/10
Early stopping triggered.
Loading best model for visualization...
Generating visualization...


/tmp/ipython-input-1775095349.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Visualization saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/prediction_visuals_resnet18_ta_0.5_tb_0.5_lre_0.001_lrd_0.001_fee_50_fte_50.png
Training Complete.


In [ ]:
# COMPLETE
main(
    encoder_name="resnet18",
    batch_size=128,
    full_training_epochs=50,
    # Change learning rate between encoder and decoder
    learning_rate=None,
    learning_rate_encoder=1e-5,
    learning_rate_decoder=1e-3
)

Setting up on cuda...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found kaggle.json in Drive at /content/drive/MyDrive/kaggle.json
[train] Final Dataset Size: 9603 images.
[test] Final Dataset Size: 1695 images.


/tmp/ipython-input-3050472091.py:96: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Checkpoints will be saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet18_ta_0.5_tb_0.5_lre_1e-05_lrd_0.001_fee_0_fte_50.pth
Starting training on cuda for 50 epochs...


  0%|          | 0/76 [00:00<?, ?it/s]/tmp/ipython-input-3144592320.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 76/76 [00:49<00:00,  1.52it/s, loss=0.734]
/tmp/ipython-input-3809429390.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/50 [Full training] | Loss: 0.7564 | Dice Score: 0.5414
New Best Score! Saved Model to /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet18_ta_0.5_tb_0.5_lre_1e-05_lrd_0.001_fee_0_fte_50.pth (Dice: 0.5414)


...

Epoch 27/50 [Full training] | Loss: 0.2746 | Dice Score: 0.7090
No improvement. Patience: 9/10


100%|██████████| 76/76 [00:51<00:00,  1.47it/s, loss=0.389]


Epoch 28/50 [Full training] | Loss: 0.2743 | Dice Score: 0.7042
No improvement. Patience: 10/10
Early stopping triggered.
Loading best model for visualization...
Generating visualization...


/tmp/ipython-input-1775095349.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Visualization saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/prediction_visuals_resnet18_ta_0.5_tb_0.5_lre_1e-05_lrd_0.001_fee_0_fte_50.png
Training Complete.


In [ ]:
# COMPLETE
main(
    encoder_name="resnet18",
    batch_size=128,
    full_training_epochs=50,
    # Change learning rate between encoder and decoder
    learning_rate=None,
    learning_rate_encoder=5e-5,
    learning_rate_decoder=5e-3
)

Setting up on cuda...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found kaggle.json in Drive at /content/drive/MyDrive/kaggle.json
[train] Final Dataset Size: 9603 images.
[test] Final Dataset Size: 1695 images.


/tmp/ipython-input-1728581891.py:96: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Checkpoints will be saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet18_ta_0.5_tb_0.5_lre_5e-05_lrd_0.005_fee_0_fte_50.pth
Starting training on cuda for 50 epochs...


  0%|          | 0/76 [00:00<?, ?it/s]/tmp/ipython-input-3144592320.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 76/76 [00:50<00:00,  1.51it/s, loss=0.555]
/tmp/ipython-input-3809429390.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/50 [Full training] | Loss: 0.7554 | Dice Score: 0.5194
New Best Score! Saved Model to /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet18_ta_0.5_tb_0.5_lre_5e-05_lrd_0.005_fee_0_fte_50.pth (Dice: 0.5194)


...

Epoch 36/50 [Full training] | Loss: 0.2481 | Dice Score: 0.7285
No improvement. Patience: 9/10


100%|██████████| 76/76 [00:51<00:00,  1.48it/s, loss=0.291]


Epoch 37/50 [Full training] | Loss: 0.2408 | Dice Score: 0.7277
No improvement. Patience: 10/10
Early stopping triggered.
Loading best model for visualization...
Generating visualization...


/tmp/ipython-input-1775095349.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Visualization saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/prediction_visuals_resnet18_ta_0.5_tb_0.5_lre_5e-05_lrd_0.005_fee_0_fte_50.png
Training Complete.


In [ ]:
# COMPLETE
main(
    encoder_name="resnet18",
    batch_size=128,
    learning_rate=1e-3,
    full_training_epochs=50,
    # Increase penalty for false positives
    tversky_alpha=0.55,
    tversky_beta=0.45
)

Setting up on cuda...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found kaggle.json in Drive at /content/drive/MyDrive/kaggle.json
[train] Final Dataset Size: 9603 images.
[test] Final Dataset Size: 1695 images.


/tmp/ipython-input-3228062457.py:96: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Checkpoints will be saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet18_ta_0.55_tb_0.45_lre_0.001_lrd_0.001_fee_0_fte_50.pth
Starting training on cuda for 50 epochs...


  0%|          | 0/76 [00:00<?, ?it/s]/tmp/ipython-input-3144592320.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 76/76 [00:50<00:00,  1.50it/s, loss=0.732]
/tmp/ipython-input-3809429390.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/50 [Full training] | Loss: 0.8524 | Dice Score: 0.4851
New Best Score! Saved Model to /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet18_ta_0.55_tb_0.45_lre_0.001_lrd_0.001_fee_0_fte_50.pth (Dice: 0.4851)


...

Epoch 43/50 [Full training] | Loss: 0.2910 | Dice Score: 0.7051
No improvement. Patience: 9/10


100%|██████████| 76/76 [00:50<00:00,  1.49it/s, loss=0.265]


Epoch 44/50 [Full training] | Loss: 0.2964 | Dice Score: 0.7117
No improvement. Patience: 10/10
Early stopping triggered.
Loading best model for visualization...
Generating visualization...


/tmp/ipython-input-1775095349.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Visualization saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/prediction_visuals_resnet18_ta_0.55_tb_0.45_lre_0.001_lrd_0.001_fee_0_fte_50.png
Training Complete.


## resnet34

In [ ]:
# COMPLETE
main(
    encoder_name="resnet34",
    # Increased Batch Size to 128 for ResNet34 (Lightest network = Lowest VRAM usage)
    batch_size=128,
    # Scaled LR up because we increased Batch Size (Linear Scaling Rule)
    learning_rate=5e-4,
    full_training_epochs=50
)

Setting up on cuda...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found kaggle.json in Drive at /content/drive/MyDrive/kaggle.json
[train] Final Dataset Size: 9603 images.
[test] Final Dataset Size: 1695 images.


/tmp/ipython-input-1728581891.py:96: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Checkpoints will be saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet34_ta_0.5_tb_0.5_lre_0.0005_lrd_0.0005_fee_0_fte_50.pth
Starting training on cuda for 50 epochs...


  0%|          | 0/76 [00:00<?, ?it/s]/tmp/ipython-input-3144592320.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 76/76 [00:55<00:00,  1.38it/s, loss=0.933]
/tmp/ipython-input-3809429390.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/50 [Full training] | Loss: 0.8400 | Dice Score: 0.4030
New Best Score! Saved Model to /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet34_ta_0.5_tb_0.5_lre_0.0005_lrd_0.0005_fee_0_fte_50.pth (Dice: 0.4030)


...

Epoch 48/50 [Full training] | Loss: 0.3260 | Dice Score: 0.5993
No improvement. Patience: 7/10


100%|██████████| 76/76 [00:56<00:00,  1.34it/s, loss=0.531]


Epoch 49/50 [Full training] | Loss: 0.3205 | Dice Score: 0.5901
No improvement. Patience: 8/10
Loading best model for visualization...
Generating visualization...


/tmp/ipython-input-1775095349.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Visualization saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/prediction_visuals_resnet34_ta_0.5_tb_0.5_lre_0.0005_lrd_0.0005_fee_0_fte_50.png
Training Complete.


In [ ]:
# COMPLETE
main(
    encoder_name="resnet34",
    batch_size=128,
    learning_rate=5e-4,
    full_training_epochs=50,
    # First train the decoder for 50 epochs
    freeze_encoder_epochs=50
)

Setting up on cuda...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found kaggle.json in Drive at /content/drive/MyDrive/kaggle.json
[train] Final Dataset Size: 9603 images.
[test] Final Dataset Size: 1695 images.


/tmp/ipython-input-1728581891.py:96: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Freezing encoder parameters for the first 50 epochs...
Checkpoints will be saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet34_ta_0.5_tb_0.5_lre_0.0005_lrd_0.0005_fee_50_fte_50.pth
Starting training on cuda for 100 epochs...


  0%|          | 0/76 [00:00<?, ?it/s]/tmp/ipython-input-3144592320.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 76/76 [00:46<00:00,  1.62it/s, loss=0.669]
/tmp/ipython-input-3809429390.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/100 [Decoder-only training] | Loss: 0.8402 | Dice Score: 0.3847
New Best Score! Saved Model to /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet34_ta_0.5_tb_0.5_lre_0.0005_lrd_0.0005_fee_50_fte_50.pth (Dice: 0.3847)


...

Epoch 83/100 [Full training] | Loss: 0.2761 | Dice Score: 0.7197
No improvement. Patience: 9/10


100%|██████████| 76/76 [00:55<00:00,  1.37it/s, loss=0.561]


Epoch 84/100 [Full training] | Loss: 0.2744 | Dice Score: 0.7195
No improvement. Patience: 10/10
Early stopping triggered.
Loading best model for visualization...
Generating visualization...


/tmp/ipython-input-1775095349.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Visualization saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/prediction_visuals_resnet34_ta_0.5_tb_0.5_lre_0.0005_lrd_0.0005_fee_50_fte_50.png
Training Complete.


In [ ]:
# COMPLETE
main(
    encoder_name="resnet34",
    batch_size=128,
    full_training_epochs=50,
    # Change learning rate between encoder and decoder
    learning_rate=None,
    learning_rate_encoder=1e-5,
    learning_rate_decoder=1e-4
)

Setting up on cuda...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found kaggle.json in Drive at /content/drive/MyDrive/kaggle.json
[train] Final Dataset Size: 9603 images.
[test] Final Dataset Size: 1695 images.


/tmp/ipython-input-1728581891.py:96: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Checkpoints will be saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet34_ta_0.5_tb_0.5_lre_1e-05_lrd_0.0001_fee_0_fte_50.pth
Starting training on cuda for 50 epochs...


  0%|          | 0/76 [00:00<?, ?it/s]/tmp/ipython-input-3144592320.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 76/76 [00:56<00:00,  1.35it/s, loss=0.865]
/tmp/ipython-input-3809429390.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/50 [Full training] | Loss: 0.8584 | Dice Score: 0.2614
New Best Score! Saved Model to /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet34_ta_0.5_tb_0.5_lre_1e-05_lrd_0.0001_fee_0_fte_50.pth (Dice: 0.2614)


...

Epoch 20/50 [Full training] | Loss: 0.6998 | Dice Score: 0.5442
No improvement. Patience: 9/10


100%|██████████| 76/76 [00:56<00:00,  1.35it/s, loss=0.891]


Epoch 21/50 [Full training] | Loss: 0.6955 | Dice Score: 0.5548
No improvement. Patience: 10/10
Early stopping triggered.
Loading best model for visualization...
Generating visualization...


/tmp/ipython-input-1775095349.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Visualization saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/prediction_visuals_resnet34_ta_0.5_tb_0.5_lre_1e-05_lrd_0.0001_fee_0_fte_50.png
Training Complete.


In [ ]:
# COMPLETE
main(
    encoder_name="resnet34",
    batch_size=128,
    full_training_epochs=50,
    # Change learning rate between encoder and decoder
    learning_rate=None,
    learning_rate_encoder=5e-5,
    learning_rate_decoder=5e-4
)

Setting up on cuda...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found kaggle.json in Drive at /content/drive/MyDrive/kaggle.json
[train] Final Dataset Size: 9603 images.
[test] Final Dataset Size: 1695 images.


/tmp/ipython-input-1728581891.py:96: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Checkpoints will be saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet34_ta_0.5_tb_0.5_lre_5e-05_lrd_0.0005_fee_0_fte_50.pth
Starting training on cuda for 50 epochs...


  0%|          | 0/76 [00:00<?, ?it/s]/tmp/ipython-input-3144592320.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 76/76 [00:55<00:00,  1.37it/s, loss=0.825]
/tmp/ipython-input-3809429390.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/50 [Full training] | Loss: 0.8105 | Dice Score: 0.5051
New Best Score! Saved Model to /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet34_ta_0.5_tb_0.5_lre_5e-05_lrd_0.0005_fee_0_fte_50.pth (Dice: 0.5051)


...

Epoch 48/50 [Full training] | Loss: 0.2617 | Dice Score: 0.7349
No improvement. Patience: 4/10


100%|██████████| 76/76 [00:57<00:00,  1.32it/s, loss=0.343]


Epoch 49/50 [Full training] | Loss: 0.2589 | Dice Score: 0.7390
New Best Score! Saved Model to /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet34_ta_0.5_tb_0.5_lre_5e-05_lrd_0.0005_fee_0_fte_50.pth (Dice: 0.7390)
Loading best model for visualization...
Generating visualization...


/tmp/ipython-input-1775095349.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Visualization saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/prediction_visuals_resnet34_ta_0.5_tb_0.5_lre_5e-05_lrd_0.0005_fee_0_fte_50.png
Training Complete.


In [ ]:
# COMPLETE
main(
    encoder_name="resnet34",
    batch_size=128,
    learning_rate=5e-4,
    full_training_epochs=50,
    # Increase penalty for false positives
    tversky_alpha=0.55,
    tversky_beta=0.45
)

Setting up on cuda...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found kaggle.json in Drive at /content/drive/MyDrive/kaggle.json
[train] Final Dataset Size: 9603 images.
[test] Final Dataset Size: 1695 images.


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/tmp/ipython-input-1728581891.py:96: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Checkpoints will be saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet34_ta_0.55_tb_0.45_lre_0.0005_lrd_0.0005_fee_0_fte_50.pth
Starting training on cuda for 50 epochs...


  0%|          | 0/76 [00:00<?, ?it/s]/tmp/ipython-input-3144592320.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 76/76 [00:55<00:00,  1.37it/s, loss=0.812]
/tmp/ipython-input-3809429390.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/50 [Full training] | Loss: 0.8025 | Dice Score: 0.5539
New Best Score! Saved Model to /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet34_ta_0.55_tb_0.45_lre_0.0005_lrd_0.0005_fee_0_fte_50.pth (Dice: 0.5539)


...

Epoch 47/50 [Full training] | Loss: 0.2835 | Dice Score: 0.7208
No improvement. Patience: 9/10


100%|██████████| 76/76 [00:56<00:00,  1.35it/s, loss=0.367]


Epoch 48/50 [Full training] | Loss: 0.2907 | Dice Score: 0.7149
No improvement. Patience: 10/10
Early stopping triggered.
Loading best model for visualization...
Generating visualization...


/tmp/ipython-input-1775095349.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Visualization saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/prediction_visuals_resnet34_ta_0.55_tb_0.45_lre_0.0005_lrd_0.0005_fee_0_fte_50.png
Training Complete.


## resnet50

In [ ]:
main(
    encoder_name="resnet50",
    # Increased Batch Size to 128 for ResNet50 (Lightest network = Lowest VRAM usage)
    batch_size=128,
    # Scaled LR up because we increased Batch Size (Linear Scaling Rule)
    learning_rate=1e-4,
    full_training_epochs=50
)

Setting up on cuda...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found kaggle.json in Drive at /content/drive/MyDrive/kaggle.json
[train] Final Dataset Size: 9603 images.
[test] Final Dataset Size: 1695 images.


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/tmp/ipython-input-1728581891.py:96: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Checkpoints will be saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet50_ta_0.5_tb_0.5_lre_0.0001_lrd_0.0001_fee_0_fte_50.pth
Starting training on cuda for 50 epochs...


  0%|          | 0/76 [00:00<?, ?it/s]/tmp/ipython-input-3144592320.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 76/76 [01:09<00:00,  1.09it/s, loss=0.77]
/tmp/ipython-input-3809429390.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/50 [Full training] | Loss: 0.8052 | Dice Score: 0.4395
New Best Score! Saved Model to /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet50_ta_0.5_tb_0.5_lre_0.0001_lrd_0.0001_fee_0_fte_50.pth (Dice: 0.4395)


...

Epoch 34/50 [Full training] | Loss: 0.5894 | Dice Score: 0.7042
No improvement. Patience: 9/10


100%|██████████| 76/76 [01:08<00:00,  1.12it/s, loss=0.755]


Epoch 35/50 [Full training] | Loss: 0.5870 | Dice Score: 0.7001
No improvement. Patience: 10/10
Early stopping triggered.
Loading best model for visualization...
Generating visualization...


/tmp/ipython-input-1775095349.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Visualization saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/prediction_visuals_resnet50_ta_0.5_tb_0.5_lre_0.0001_lrd_0.0001_fee_0_fte_50.png
Training Complete.


In [ ]:
main(
    encoder_name="resnet50",
    batch_size=128,
    learning_rate=1e-4,
    full_training_epochs=50,
    # First train the decoder for 50 epochs
    freeze_encoder_epochs=50
)

Setting up on cuda...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found kaggle.json in Drive at /content/drive/MyDrive/kaggle.json
[train] Final Dataset Size: 9603 images.
[test] Final Dataset Size: 1695 images.


/tmp/ipython-input-1728581891.py:96: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Freezing encoder parameters for the first 50 epochs...
Checkpoints will be saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet50_ta_0.5_tb_0.5_lre_0.0001_lrd_0.0001_fee_50_fte_50.pth
Starting training on cuda for 100 epochs...


  0%|          | 0/76 [00:00<?, ?it/s]/tmp/ipython-input-3144592320.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 76/76 [00:52<00:00,  1.46it/s, loss=0.904]
/tmp/ipython-input-3809429390.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/100 [Decoder-only training] | Loss: 0.8848 | Dice Score: 0.2871
New Best Score! Saved Model to /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet50_ta_0.5_tb_0.5_lre_0.0001_lrd_0.0001_fee_50_fte_50.pth (Dice: 0.2871)


...

Epoch 69/100 [Full training] | Loss: 0.7726 | Dice Score: 0.4616
No improvement. Patience: 9/10


100%|██████████| 76/76 [01:11<00:00,  1.07it/s, loss=0.782]


Epoch 70/100 [Full training] | Loss: 0.7700 | Dice Score: 0.4880
No improvement. Patience: 10/10
Early stopping triggered.
Loading best model for visualization...
Generating visualization...


/tmp/ipython-input-1775095349.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Visualization saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/prediction_visuals_resnet50_ta_0.5_tb_0.5_lre_0.0001_lrd_0.0001_fee_50_fte_50.png
Training Complete.


In [ ]:
main(
    encoder_name="resnet50",
    batch_size=128,
    full_training_epochs=50,
    # Change learning rate between encoder and decoder
    learning_rate=None,
    learning_rate_encoder=1e-5,
    learning_rate_decoder=1e-4
)

Setting up on cuda...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found kaggle.json in Drive at /content/drive/MyDrive/kaggle.json
[train] Final Dataset Size: 9603 images.
[test] Final Dataset Size: 1695 images.


/tmp/ipython-input-1728581891.py:96: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Checkpoints will be saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet50_ta_0.5_tb_0.5_lre_1e-05_lrd_0.0001_fee_0_fte_50.pth
Starting training on cuda for 50 epochs...


  0%|          | 0/76 [00:00<?, ?it/s]/tmp/ipython-input-3144592320.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 76/76 [01:16<00:00,  1.00s/it, loss=0.859]
/tmp/ipython-input-3809429390.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/50 [Full training] | Loss: 0.8716 | Dice Score: 0.3230
New Best Score! Saved Model to /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet50_ta_0.5_tb_0.5_lre_1e-05_lrd_0.0001_fee_0_fte_50.pth (Dice: 0.3230)


...

Epoch 28/50 [Full training] | Loss: 0.7978 | Dice Score: 0.5312
No improvement. Patience: 9/10


100%|██████████| 76/76 [01:16<00:00,  1.00s/it, loss=0.811]


Epoch 29/50 [Full training] | Loss: 0.7977 | Dice Score: 0.5124
No improvement. Patience: 10/10
Early stopping triggered.
Loading best model for visualization...
Generating visualization...


/tmp/ipython-input-1775095349.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Visualization saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/prediction_visuals_resnet50_ta_0.5_tb_0.5_lre_1e-05_lrd_0.0001_fee_0_fte_50.png
Training Complete.


In [ ]:
main(
    encoder_name="resnet50",
    batch_size=128,
    full_training_epochs=50,
    # Change learning rate between encoder and decoder
    learning_rate=None,
    learning_rate_encoder=5e-5,
    learning_rate_decoder=5e-4
)

Setting up on cuda...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found kaggle.json in Drive at /content/drive/MyDrive/kaggle.json
[train] Final Dataset Size: 9603 images.
[test] Final Dataset Size: 1695 images.


/tmp/ipython-input-1728581891.py:96: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Checkpoints will be saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet50_ta_0.5_tb_0.5_lre_5e-05_lrd_0.0005_fee_0_fte_50.pth
Starting training on cuda for 50 epochs...


  0%|          | 0/76 [00:00<?, ?it/s]/tmp/ipython-input-3144592320.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 76/76 [01:17<00:00,  1.02s/it, loss=0.779]
/tmp/ipython-input-3809429390.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/50 [Full training] | Loss: 0.7961 | Dice Score: 0.4913
New Best Score! Saved Model to /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet50_ta_0.5_tb_0.5_lre_5e-05_lrd_0.0005_fee_0_fte_50.pth (Dice: 0.4913)


...

Epoch 48/50 [Full training] | Loss: 0.2491 | Dice Score: 0.7391
No improvement. Patience: 4/10


100%|██████████| 76/76 [01:16<00:00,  1.01s/it, loss=0.373]


Epoch 49/50 [Full training] | Loss: 0.2478 | Dice Score: 0.7392
No improvement. Patience: 5/10
Loading best model for visualization...
Generating visualization...


/tmp/ipython-input-1775095349.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Visualization saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/prediction_visuals_resnet50_ta_0.5_tb_0.5_lre_5e-05_lrd_0.0005_fee_0_fte_50.png
Training Complete.


In [ ]:
main(
    encoder_name="resnet50",
    batch_size=128,
    learning_rate=1e-4,
    full_training_epochs=50,
    # Increase penalty for false positives
    tversky_alpha=0.55,
    tversky_beta=0.45
)

Setting up on cuda...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found kaggle.json in Drive at /content/drive/MyDrive/kaggle.json
[train] Final Dataset Size: 9603 images.
[test] Final Dataset Size: 1695 images.


/tmp/ipython-input-1728581891.py:96: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Checkpoints will be saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet50_ta_0.55_tb_0.45_lre_0.0001_lrd_0.0001_fee_0_fte_50.pth
Starting training on cuda for 50 epochs...


  0%|          | 0/76 [00:00<?, ?it/s]/tmp/ipython-input-3144592320.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 76/76 [01:14<00:00,  1.02it/s, loss=0.874]
/tmp/ipython-input-3809429390.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/50 [Full training] | Loss: 0.8641 | Dice Score: 0.4061
New Best Score! Saved Model to /content/drive/MyDrive/CrackSegmentation_Checkpoints/best_crack_model_resnet50_ta_0.55_tb_0.45_lre_0.0001_lrd_0.0001_fee_0_fte_50.pth (Dice: 0.4061)


...

Epoch 47/50 [Full training] | Loss: 0.6974 | Dice Score: 0.6949
No improvement. Patience: 9/10


100%|██████████| 76/76 [01:17<00:00,  1.02s/it, loss=0.8]


Epoch 48/50 [Full training] | Loss: 0.6948 | Dice Score: 0.6930
No improvement. Patience: 10/10
Early stopping triggered.
Loading best model for visualization...
Generating visualization...


/tmp/ipython-input-1775095349.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Visualization saved to: /content/drive/MyDrive/CrackSegmentation_Checkpoints/prediction_visuals_resnet50_ta_0.55_tb_0.45_lre_0.0001_lrd_0.0001_fee_0_fte_50.png
Training Complete.


## resnet101 (TBD)

In [ ]:
main(
    encoder_name="resnet101",
    # Reduced Batch Size to 64 for ResNet101 (Deeper network = More VRAM usage)
    batch_size=64,
    # Scaled LR down because we decreased Batch Size (Linear Scaling Rule)
    learning_rate=1e-4,
    full_training_epochs=50
)

In [ ]:
main(
    encoder_name="resnet101",
    batch_size=64,
    learning_rate=1e-4,
    full_training_epochs=50,
    # First train the decoder for 50 epochs
    freeze_encoder_epochs=50
)

In [ ]:
main(
    encoder_name="resnet101",
    batch_size=64,
    full_training_epochs=50,
    # Change learning rate between encoder and decoder
    learning_rate=None,
    learning_rate_encoder=1e-5,
    learning_rate_decoder=1e-4
)

In [ ]:
main(
    encoder_name="resnet101",
    batch_size=64,
    full_training_epochs=50,
    # Change learning rate between encoder and decoder
    learning_rate=None,
    learning_rate_encoder=5e-5,
    learning_rate_decoder=5e-4
)

In [ ]:
main(
    encoder_name="resnet101",
    batch_size=64,
    learning_rate=1e-4,
    full_training_epochs=50,
    # Increase penalty for false positives
    tversky_alpha=0.55,
    tversky_beta=0.45
)

## resnet152 (TBD)

In [ ]:
main(
    encoder_name="resnet152",
    # Reduced Batch Size to 32 for ResNet152 (Deepest network = Highest VRAM usage)
    batch_size=32,
    # Scaled LR down because we decreased Batch Size (Linear Scaling Rule)
    learning_rate=1e-4,
    full_training_epochs=50
)

In [ ]:
main(
    encoder_name="resnet152",
    batch_size=32,
    learning_rate=1e-4,
    full_training_epochs=50,
    # First train the decoder for 50 epochs
    freeze_encoder_epochs=50
)

In [ ]:
main(
    encoder_name="resnet152",
    batch_size=32,
    full_training_epochs=50,
    # Change learning rate between encoder and decoder
    learning_rate=None,
    learning_rate_encoder=1e-5,
    learning_rate_decoder=1e-4
)

In [ ]:
main(
    encoder_name="resnet152",
    batch_size=32,
    full_training_epochs=50,
    # Change learning rate between encoder and decoder
    learning_rate=None,
    learning_rate_encoder=5e-5,
    learning_rate_decoder=5e-4
)

In [ ]:
main(
    encoder_name="resnet152",
    batch_size=32,
    learning_rate=1e-4,
    full_training_epochs=50,
    # Increase penalty for false positives
    tversky_alpha=0.55,
    tversky_beta=0.45
)